In [1]:
import numpy as np
import matplotlib.pyplot as plt
import legwork as lw
import astropy.units as u
from scipy.interpolate import interp1d
from scipy.integrate import trapezoid, cumulative_trapezoid
from astropy.cosmology import Planck18, z_at_value
from scipy.integrate import trapezoid
import paths
import deepdish as dd
from matplotlib import colors
from schwimmbad import MultiPool
import tqdm
import utils
import paths

In [2]:
plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 12

# update various fontsizes to match
params = {'figure.figsize': (6,4),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.7 * fs,
          'ytick.labelsize': 0.7 * fs}
plt.rcParams.update(params)


# set up the LISA frequency grid
f_LISA = np.logspace(-1, -5, 150) * u.Hz

# set up the LIGO eccentricity range
e_LIGO = np.logspace(-6, np.log10(0.0005), 10)
e_LIGO = np.append(0, e_LIGO)
e_LIGO_round = np.array([f"{e:.2e}" for e in e_LIGO])


# get the mass, mass ratio, and rate grids
down_samp_fac=20
mass_1, mass_ratio, M1, Q, dN_dm1dqdVcdt = utils.get_LIGO_rate(down_samp_fac=down_samp_fac)

mass_1 = mass_1
mass_ratio = mass_ratio
MM, QQ, EE_LIGO, FF = np.meshgrid(mass_1, mass_ratio, e_LIGO, f_LISA, indexing='ij')

In [ ]:
dat_in = list(zip(EE_LIGO.flatten(), FF.flatten(), MM.flatten(), QQ.flatten()*MM.flatten()))

with MultiPool(processes=128) as pool:
    dat_out = list(tqdm.tqdm(pool.imap(utils.get_e_LISA_t_LIGO, dat_in), total=len(dat_in)))
    
EE_LISA, TT_LIGO = zip(*dat_out)

EE_LISA = np.array(EE_LISA).reshape(FF.shape)
TT_LIGO = np.array(TT_LIGO).reshape(FF.shape) * u.yr

np.save(paths.data / 't_merge', TT_LIGO.value)
np.save(paths.data / 'e_LISA', EE_LISA)

 75%|███████▌  | 1548319/2062500 [42:24<15:16, 561.11it/s]  

In [ ]:
EE_LISA = np.load(paths.data / 'e_LISA.npy')

def chunk_list(long_list, num_chunks):
    avg = len(long_list) / float(num_chunks)
    chunks = []
    last = 0.0

    while last < len(long_list):
        chunks.append(long_list[int(last):int(last + avg)])
        last += avg

    return chunks


num_chunks = 10

snr_thresh = 12
dat_in = list(zip(MM.flatten(), QQ.flatten(), EE_LISA.flatten(), FF.flatten(), snr_thresh * np.ones(len(MM.flatten()))))

chunked_list = chunk_list(dat_in, num_chunks)
for ii, chunk in enumerate(chunked_list[3:]):
    print('running chunk: ' + str(ii+3))
    with MultiPool(processes=128) as pool:
        dat_out = list(tqdm.tqdm(pool.imap(utils.get_Vc_Dh, chunk), total=len(chunk)))
        DH, VC = zip(*dat_out)
        DH = np.array(DH)
        VC = np.array(VC)
        
        np.save(paths.data / f'comoving_volume_{ii+3}', VC)
        np.save(paths.data / f'horizon_distance_{ii+3}', DH)

In [ ]:


plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.rm'] = 'serif'
plt.rcParams['mathtext.it'] = 'serif:italic'
plt.rcParams['mathtext.bf'] = 'serif:bold'
fs = 12

# update various fontsizes to match

params = {'figure.figsize': (6,4),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.7 * fs,
          'ytick.labelsize': 0.7 * fs}
plt.rcParams.update(params)


# set up the LISA frequency grid
f_LISA = np.logspace(-1, -5, 150) * u.Hz

# set up the LIGO eccentricity range
e_LIGO = np.logspace(-6, -3, 10)
e_LIGO = np.append(0, e_LIGO)
e_LIGO_round = np.array([f"{e:.2e}" for e in e_LIGO])


# get the mass, mass ratio, and rate grids
down_samp_fac=10
mass_1, mass_ratio, M1, Q, dN_dm1dqdVcdt = utils.get_LIGO_rate(down_samp_fac=down_samp_fac)

mass_1 = mass_1
mass_ratio = mass_ratio
MM, QQ, EE_LIGO, FF = np.meshgrid(mass_1, mass_ratio, e_LIGO, f_LISA, indexing='ij')

In [ ]:
MM.flatten().shape

In [ ]:
plt.plot(mass_1, dN_dm1dqdVcdt[-1,:])
plt.yscale('log')

In [ ]:
#
#
#dat_in = list(zip(EE_LIGO.flatten(), FF.flatten(), MM.flatten(), QQ.flatten()*MM.flatten()))
#
#with MultiPool(processes=96) as pool:
#    dat_out = list(tqdm.tqdm(pool.imap(utils.get_e_LISA_t_LIGO, dat_in), total=len(dat_in)))
#    
#EE_LISA, TT_LIGO = zip(*dat_out)
#
#EE_LISA = np.array(EE_LISA).reshape(FF.shape)
#TT_LIGO = np.array(TT_LIGO).reshape(FF.shape) * u.yr
#
#np.save(paths.data / 't_merge', TT_LIGO.value)
#np.save(paths.data / 'e_LISA', EE_LISA)
#

In [ ]:
TT_LIGO = np.load(paths.data / 't_merge.npy')*u.Gpc
EE_LISA = np.load(paths.data / 'e_LISA.npy')

In [ ]:
#snr_thresh = 12
#dat_in = list(zip(MM.flatten(), QQ.flatten(), EE_LISA.flatten(), FF.flatten(), snr_thresh * np.ones(len(MM.flatten()))))
#with MultiPool(processes=92) as pool:
#    dat_out = list(tqdm.tqdm(pool.imap(utils.get_Vc_Dh, dat_in), total=len(dat_in)))
#    
#DH, VC = zip(*dat_out)
#DH = np.array(DH).reshape(QQ.shape) * u.Gpc
#VC = np.array(VC).reshape(QQ.shape) * u.Gpc**3
#
#np.save(paths.data / 'comoving_volume', VC.value)
#np.save(paths.data / 'horizon_distance', DH.value)
#
#

In [ ]:
VC = np.load(paths.data / 'comoving_volume.npy')*u.Gpc**3
DH = np.load(paths.data / 'horizon_distance.npy')*u.Gpc

In [ ]:
dT_LIGO_df_LISA = utils.dTmerger_df(MM, QQ*MM, FF, EE_LISA).to(u.yr / u.Hz)


In [ ]:
import cmasher as cmr

In [ ]:
cs = cmr.take_cmap_colors('cmr.dusk', len(mass_1), cmap_range=(0.15, 0.9), return_fmt='hex')

In [ ]:
mass_1

In [ ]:
mass_ratio, e_LIGO[15]

In [ ]:
np.shape(DH)

In [ ]:
ind_m_10 = 2
ind_m_35 = 7
ind_m_80 = 16
ind_q05 = 4
ind_q09 = 9
ind_circ = 0
ind_ecc_mid = 7
ind_ecc_high = 15

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14,3))
ax1.plot(f_LISA.value, DH[ind_m_10,ind_q09,ind_circ,:].value, ls='-', color=cs[ind_m_10], zorder=10, label=r'$M_1=10\,M_{\odot}$')
ax2.plot(f_LISA.value, DH[ind_m_10,ind_q09,ind_ecc_mid,:].value, ls='--', color=cs[ind_m_10], zorder=10, label=r'$e_{\rm{LIGO}}=10^{-5}$')
ax3.plot(f_LISA.value, DH[ind_m_10,ind_q09,ind_ecc_high,:].value, ls=':', color=cs[ind_m_10], zorder=10, label=r'$e_{\rm{LIGO}}=10^{-3}$')
#plt.fill_between(f_LISA.value, DH[ind_m_10,ind_q09,ind_circ,:].value, DH[ind_m_10,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_10], alpha=0.5, zorder=10)

ax1.plot(f_LISA.value, DH[ind_m_35,ind_q09,ind_circ,:].value, ls='-', color=cs[ind_m_35], zorder=5, label=r'$M_1=35\,M_{\odot}$')
ax2.plot(f_LISA.value, DH[ind_m_35,ind_q09,ind_ecc_mid,:].value, ls='--', color=cs[ind_m_35], zorder=5)
ax3.plot(f_LISA.value, DH[ind_m_35,ind_q09,ind_ecc_high,:].value, ls=':', color=cs[ind_m_35], zorder=5)
#plt.fill_between(f_LISA.value, DH[ind_m_35,ind_q09,ind_circ,:].value, DH[ind_m_35,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_35], alpha=0.5, zorder=5)


ax1.plot(f_LISA.value, DH[ind_m_80,ind_q09,ind_circ,:].value, ls='-', color=cs[ind_m_80], label=r'$M_1=80\,M_{\odot}$')
ax2.plot(f_LISA.value, DH[ind_m_80,ind_q09,ind_ecc_mid,:].value, ls='--', color=cs[ind_m_80])
ax3.plot(f_LISA.value, DH[ind_m_80,ind_q09,ind_ecc_high,:].value, ls=':', color=cs[ind_m_80])
#plt.fill_between(f_LISA.value, DH[ind_m_80,ind_q09,ind_circ,:].value, DH[ind_m_80,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_80], alpha=0.5)

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim(1e-5, 3)
ax1.set_xlim(1e-5, 1e-1)
ax1.set_xlabel(r'$f_{\rm{LISA}}$ [Hz]')

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_ylim(1e-5, 3)
ax2.set_xlim(1e-5, 1e-1)
ax2.set_xlabel(r'$f_{\rm{LISA}}$ [Hz]')

ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_ylim(1e-5, 3)
ax3.set_xlim(1e-5, 1e-1)
ax3.set_xlabel(r'$f_{\rm{LISA}}$ [Hz]')

ax1.legend(loc='upper left', frameon=False, prop={'size':11})
ax2.legend(loc='upper left', frameon=False, prop={'size':11})
ax3.legend(loc='upper left', frameon=False, prop={'size':11})

ax1.set_ylabel('horizon distance [Gpc]')

In [ ]:
ind_m_10 = 2
ind_m_35 = 7
ind_m_80 = 16
ind_q05 = 4
ind_q09 = 9
ind_circ = 0
ind_ecc_mid = 7
ind_ecc_high = 15

fig, (ax1) = plt.subplots(1, 1, figsize=(4,3))
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_10,ind_q09,ind_circ,:].to(u.yr/u.Hz).value, ls='-', color=cs[ind_m_10], zorder=10, label=r'$e_{\rm{LIGO}}=0$')
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_10,ind_q09,ind_ecc_mid,:].to(u.yr/u.Hz).value, ls='--', color=cs[ind_m_10], zorder=10, label=r'$e_{\rm{LIGO}}=10^{-5}$')
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_10,ind_q09,ind_ecc_high,:].to(u.yr/u.Hz).value, ls=':', color=cs[ind_m_10], zorder=10, label=r'$e_{\rm{LIGO}}=10^{-3}$')
#plt.fill_between(f_LISA.value, DH[ind_m_10,ind_q09,ind_circ,:].value, DH[ind_m_10,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_10], alpha=0.5, zorder=10)

ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_35,ind_q09,ind_circ,:].to(u.yr/u.Hz).value, ls='-', color=cs[ind_m_35], zorder=5)#
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_35,ind_q09,ind_ecc_mid,:].to(u.yr/u.Hz).value, ls='--', color=cs[ind_m_35], zorder=5)
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_35,ind_q09,ind_ecc_high,:].to(u.yr/u.Hz).value, ls=':', color=cs[ind_m_35], zorder=5)
#plt.fill_between(f_LISA.value, DH[ind_m_35,ind_q09,ind_circ,:].value, DH[ind_m_35,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_35], alpha=0.5, zorder=5)


ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_80,ind_q09,ind_circ,:].to(u.yr/u.Hz).value, ls='-', color=cs[ind_m_80])
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_80,ind_q09,ind_ecc_mid,:].to(u.yr/u.Hz).value, ls='--', color=cs[ind_m_80])
ax1.plot(f_LISA.value, dT_LIGO_df_LISA[ind_m_80,ind_q09,ind_ecc_high,:].to(u.yr/u.Hz).value, ls=':', color=cs[ind_m_80])
#plt.fill_between(f_LISA.value, DH[ind_m_80,ind_q09,ind_circ,:].value, DH[ind_m_80,ind_q09,ind_ecc_high,:].value, color=cs[ind_m_80], alpha=0.5)

ax1.set_xscale('log')
ax1.set_yscale('log')
#ax1.set_ylim(1e-5, 3)
ax1.set_xlim(1e-5, 1e-1)
ax1.set_xlabel(r'$f_{\rm{LISA}}$ [Hz]')

ax1.legend(loc='upper right', frameon=False, prop={'size':11})

ax1.set_ylabel(r'$\partial T_{\rm{merge}}(e_{\rm{LISA}},f_{\rm{LISA}})/\partial f_{\rm{LISA}}$')

In [ ]:
LISA_norm = dT_LIGO_df_LISA * VC

In [ ]:
np.shape(LISA_norm), np.shape(dN_dm1dqdVcdt)

In [ ]:
N_LISA_cumulative = np.zeros((len(mass_1), len(mass_ratio), len(e_LIGO), len(f_LISA)))
N_LISA = np.zeros((len(mass_1), len(mass_ratio), len(e_LIGO)))
for ee, ecc in enumerate(e_LIGO):
    for mm, m in enumerate(mass_1):
        for qq, q in enumerate(mass_ratio):
            arg = (LISA_norm[mm, qq, ee,:] * dN_dm1dqdVcdt[qq,mm]).to(u.Hz**(-1) / u.Msun)
            N_LISA_cumulative[mm,qq,ee,:] = cumulative_trapezoid(arg, -f_LISA, initial=0)
            N_LISA[mm,qq,ee] = trapezoid(arg, -f_LISA).value
            


In [ ]:
M1.shape, Q

In [ ]:
plt.scatter(M1[:,:], Q[:,:], c=N_LISA[:,:,0].T, s=10, norm=colors.LogNorm(vmin=1e-5, vmax=0.4))
plt.colorbar()

In [ ]:
plt.scatter(M1[:,:], Q[:,:], c=N_LISA[:,:,7].T, s=10, norm=colors.LogNorm(vmin=1e-5, vmax=0.4))
plt.colorbar()

In [ ]:
plt.scatter(M1[:,:], Q[:,:], c=N_LISA[:,:,15].T, s=10, norm=colors.LogNorm(vmin=1e-5, vmax=0.4))
plt.colorbar()

In [ ]:
dN_dm1_cumulative_list = []
for ee, ecc in enumerate(e_LIGO):
    dN_dm1_cumulative = np.zeros((len(mass_1),len(f_LISA)))
    for mm, m in enumerate(mass_1):
        dN_dm1_cumulative[mm,:] = trapezoid(N_LISA_cumulative[mm,:,ee,:], mass_ratio, axis=0)
    dN_dm1_cumulative_list.append(dN_dm1_cumulative)

In [ ]:
def format_sci_notation(num):
    exponent = int(np.floor(np.log10(num)))
    coefficient = num / 10**exponent
    return rf'${coefficient:.1f} \times 10^{{{exponent}}}$'
dN_dm1_list = []
for ii in range(0,len(e_LIGO)):
    dN_dm1 = []
    for mm, m in enumerate(mass_1):
        dN_dm1.append(np.max(dN_dm1_cumulative_list[ii][mm,:]))
    if ii==0:
        plt.plot(mass_1, dN_dm1, label=rf'$e_{{\rm{{LIGO}}}}={e_LIGO[ii]}$', c=cs[ii])
    else:
        plt.plot(mass_1, dN_dm1, label=rf'$e_{{\rm{{LIGO}}}}$={format_sci_notation(e_LIGO[ii])}', c=cs[ii])
    dN_dm1_list.append(dN_dm1)
plt.legend()
plt.yscale('log')


In [ ]:
for ii, n in enumerate(dN_dm1_list):
    print(e_LIGO[ii], trapezoid(n/u.Msun, mass_1))